## Analyse rail stations and airports in NUTS to get rail stations which are more 'important'

Importance is defined as rail stations within the NUTS which connect with other NUTS regions. This is done by:


<ol>
  <p>For each rail station compute:</p>
  <li>The NUTS (level 3) that are reachable from that station.</li>
  <li>The number of services in the GTFS data for each NUTS reachable.</li>
  <li>The total number of trips from that station</li>
  <li>The distance of the station to the centroid of the population of the NUTS-3 where the rail station is located</li>
    
</ol>


<ol>
  <p>Then, for each NUTS-3 remove the rail stations ‘dominated’ by other stations. This is done by considering that a rail station is dominated by another. Station A is dominated by B, and therefore not considered if:</p>
  <li>All the reachable NUTS from A are also reachable from B and with more services in B.</li>
  <li>All the reachable NUTS from A are also reachable from B with the same number of services to each NUTS but B is closer to the centre of the polygon defining the NUTS-3.</li>    
</ol>

Note, if a station has a parent station that parent is used.

## Input data and parameters

In [ ]:
nuts_level = 3
threshold_distance_plot_between_nuts = 400

nuts_data_path = '../data/EUROSTAT/NUTS_RG_01M_2021_4326.shp'
countries = ['ES', 'DE']
#gtfs_paths = [{'country':'ES','path':'../data/gtfs/gtfs_es/'},
#              {'country':'DE','path':'../data/gtfs/gtfs_de/'}]

gtfs_paths = [{'country':'ES','path':'../data/gtfs/gtfs_es_20220708/'},
              {'country':'DE','path':'../data/gtfs/gtfs_de/'}]
flight_schedules_path = '../data/flight_schedules/flight_schedule_sample.parquet'
airports_path = '../data/airports/airport_info_static_old1409.parquet'
census_data_path = '../data/EUROSTAT/Eurostat_Census-GRID_2021_V1-0/ESTAT_OBS-VALUE-T_2021_V1-0.tiff'

output_path = '../output/data_analysis_GTFS_2022/'
output_path_figures = '../output/data_analysis_GTFS_2022/figs/'
generate_plots = False

# Assuming the correct CRS for the data is EPSG:4326
crs_stops = 'EPSG:4326'
crs_airports = 'EPSG:4326'
crs_nuts = 'EPSG:4326'


## Processing

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# Load NUTS boundary data
nuts_data = gpd.read_file(nuts_data_path)

# Load rail data
l_stops = []
parent_stop_dict = {}
for gtfs in gtfs_paths:
    stops = pd.read_csv(gtfs['path']+'stops.txt')
    stops['stop_id'] = stops['stop_id'].apply(lambda x: gtfs['country']+str(x))
    stops['country'] = gtfs['country']
    # Create a dictionary of parent stations
    parent_stop_dict_country = {row['stop_id']: f"{gtfs['country']}{int(row['parent_station'])}" 
                                for index, row in stops[~stops.parent_station.isnull()][['stop_id', 'parent_station']].iterrows()}

    parent_stop_dict = {**parent_stop_dict, **parent_stop_dict_country}
    
    # Drop stops that have a parent
    stops = stops[stops.parent_station.isnull()]

    l_stops += [stops]

stops = pd.concat(l_stops).reset_index(drop=True)


# Load airports data
airports = pd.read_parquet(airports_path)[['icao_id','lat','lon']]


# Prepare the data

# Create Point objects from coordinates for train stations
stops['geometry'] = stops.apply(lambda x: Point(x.stop_lon, x.stop_lat), axis=1)

# Create Point objects from coordinates for airports
airports['geometry'] = airports.apply(lambda x: Point(x.lon, x.lat), axis=1)

# Convert to GeoDataFrame
stops = gpd.GeoDataFrame(stops, geometry='geometry')
airports = gpd.GeoDataFrame(airports, geometry='geometry')

# Add CRS
stops.crs = crs_stops
airports.crs = crs_airports
nuts_data.crs = crs_nuts

# Re-project to a projected CRS (e.g., EPSG:3857)
projected_crs = 'EPSG:3857'
stops_proj = stops.to_crs(projected_crs)

# Buffer the points slightly (e.g., by 10 meters) -- due to Dagebüll Mole rail station in edge of NUTS
stops_proj['geometry'] = stops_proj['geometry'].buffer(10)

# Re-project back to the original geographic CRS
stops = stops_proj.to_crs(crs_stops)

# Perform spatial join
joined_rail = gpd.sjoin(stops, nuts_data, predicate='intersects', how='left') 
joined_airports = gpd.sjoin(airports, nuts_data, predicate='within', how='left')

# Group the rail stations by NUTS region
grouped_rail = joined_rail.groupby('NUTS_ID')[['stop_id', 'stop_name']].apply(lambda x: list(zip(x['stop_id'], x['stop_name']))).reset_index()
grouped_rail.columns = ['NUTS_ID', 'rail_stations']

grouped_airports = joined_airports.groupby('NUTS_ID')[['icao_id']].apply(lambda x: list(x['icao_id'])).reset_index()
grouped_airports.columns = ['NUTS_ID', 'airports']


# Merge the grouped data back to nuts_data
nuts_data = nuts_data.merge(grouped_rail, on='NUTS_ID', how='left')
nuts_data = nuts_data.merge(grouped_airports, on='NUTS_ID', how='left')

nuts_data['num_rail_stations'] = nuts_data['rail_stations'].apply(lambda x: len(x) if isinstance(x, list) else 0)
nuts_data['num_airports'] = nuts_data['airports'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Create a dictionary of nuts for the different rail stations and airports
dict_stations_nuts = {}
for i, row in nuts_data[(nuts_data['rail_stations'].notna()) & (nuts_data['LEVL_CODE']==nuts_level)].iterrows():
    for s in row['rail_stations']:
        dict_stations_nuts[s[0]] = row['NUTS_ID']

dict_airports_nuts = {}
for i, row in nuts_data[(nuts_data['airports'].notna()) & (nuts_data['LEVL_CODE']==nuts_level)].iterrows():
    for s in row['airports']:
        dict_airports_nuts[s] = row['NUTS_ID']


# Add the nuts to the rail stops and the airports
stops['nuts'] = stops['stop_id'].apply(lambda x: dict_stations_nuts.get(x))
airports['nuts'] = airports['icao_id'].apply(lambda x: dict_airports_nuts.get(x))

# Merge nuts into rail stops
stops = stops.merge(nuts_data[['NUTS_ID', 'geometry']], how='left', left_on=['nuts'], right_on=['NUTS_ID'], suffixes=('','_nuts'))



In [ ]:
# Save raw results (no filtering)
for c in countries:
    nuts_data[(nuts_data.CNTR_CODE==c) & (nuts_data.LEVL_CODE==3)][['NUTS_ID','LEVL_CODE','NAME_LATN','num_rail_stations','num_airports','airports','rail_stations']].to_csv(output_path+'nuts3_'+c+'_no_filtering.csv', index=False)
    nuts_data[(nuts_data.CNTR_CODE==c) & (nuts_data.LEVL_CODE==2)][['NUTS_ID','LEVL_CODE','NAME_LATN','num_rail_stations','num_airports','airports','rail_stations']].to_csv(output_path+'nuts2_'+c+'_no_filtering.csv', index=False)
    

### Filter rail stations

In [ ]:
# Compute centroid of the population for each nuts
import fiona
import rasterio
from rasterio.mask import mask
from rasterio.warp import transform_geom
import numpy as np
import matplotlib.pyplot as plt
import rasterio.warp
import pyproj
from shapely.geometry import Polygon, MultiPolygon

dict_centroid_pop = {}

# Read the NUTS region geometry
for i, row in nuts_data[(nuts_data.CNTR_CODE.isin(countries)) & (nuts_data.LEVL_CODE==nuts_level)].iterrows():
    nuts_interest_id = row.NUTS_ID
    nuts_interest_name = row.NAME_LATN
    nuts_interest = row.geometry
      
    # Read the population distribution raster
    with rasterio.open(census_data_path) as src:
        # Reproject the NUTS region geometry to match the CRS of the raster
        census_crs = src.crs
        if crs_nuts != census_crs:
            nut_region = transform_geom(
                crs_nuts, census_crs, nuts_interest, precision=6)
        # Clip the raster by the NUTS boundary
        out_image, out_transform = mask(src, [nut_region], crop=True)
        out_meta = src.meta.copy()


    # Summarize population within the NUTS region
    total_population = out_image.sum()

    # Compute the weighted sum of the population within each pixel
    weighted_population = np.sum(out_image)
    
    # Create arrays of indices along the x and y axes
    x_indices = np.arange(out_image.shape[2])
    y_indices = np.arange(out_image.shape[1])
    
    # Compute the weighted centroid
    weighted_centroid_x = (np.sum(out_image * x_indices) / weighted_population) / out_image.shape[2]
    weighted_centroid_y = (np.sum(out_image * y_indices[:, np.newaxis]) / weighted_population) / out_image.shape[1]
    
    # Get the affine transformation matrix
    transform_matrix = out_transform
    
    # Calculate the geographic coordinates of the centroid
    centroid_x_geo = transform_matrix[2] + weighted_centroid_x * out_image.shape[2] * transform_matrix[0] 
    centroid_y_geo = transform_matrix[5] + weighted_centroid_y * out_image.shape[1] * transform_matrix[4] 
    
    # Transform the centroid coordinates to the CRS of the NUTS region
    transformer = pyproj.Transformer.from_crs(census_crs, crs_nuts, always_xy=True)
    centroid_x_nut, centroid_y_nut =  transformer.transform(centroid_x_geo, centroid_y_geo)

    dict_centroid_pop[nuts_interest_id] = (centroid_x_nut, centroid_y_nut)

    if generate_plots:
        
        # Define the target CRS (e.g., EPSG:3035)
        target_crs = crs_nuts 
        
        # Create an empty array for the destination
        out_image_reprojected = np.empty((out_image.shape[0], out_image.shape[1], out_image.shape[2]), dtype=out_image.dtype)
        
        # Reproject the image to the target CRS
        out_image_reprojected, out_transform_reprojected = rasterio.warp.reproject(
            out_image,
            out_image_reprojected,
            src_crs=census_crs, 
            dst_crs=target_crs, 
            src_transform=out_transform, 
            src_nodata=None, 
            dst_width=out_image.shape[2], 
            dst_height=out_image.shape[1], 
            dst_nodata=None, 
            resampling=rasterio.enums.Resampling.nearest)
        
        
        # Calculate the extent of the reprojected image
        x_min = out_transform_reprojected[2]
        y_min = out_transform_reprojected[5] + out_transform_reprojected[4] * out_image.shape[1]
        x_max = out_transform_reprojected[2] + out_transform_reprojected[0] * out_image.shape[2]
        y_max = out_transform_reprojected[5]
        
        # Visualize the population distribution in the target CRS
        fig, ax = plt.subplots(figsize=(10, 10))
        plt.imshow(np.squeeze(out_image_reprojected), cmap='viridis', extent=[x_min, x_max, y_min, y_max],alpha=0.5)
        # Plot the NUTS geometry perimeter
        if isinstance(nuts_interest, MultiPolygon):
            for geom in nuts_interest.geoms:
                x, y = geom.exterior.xy
                ax.plot(x, y, color='blue', label='NUTS Perimeter')
        elif isinstance(nuts_interest, Polygon):
            x, y = nuts_interest.exterior.xy
            ax.plot(x, y, color='blue', label='NUTS Perimeter')
        
        ax.scatter(centroid_x_nut, centroid_y_nut, color='red', marker='x', s=100, label='Weighted Centroid')
        
        plt.colorbar(label='Population')
        plt.title('Population Distribution '+nuts_interest_name+' '+str(total_population)+' Pop')
        plt.xlabel('X Coordinate (Projected)')
        plt.ylabel('Y Coordinate (Projected)')
        plt.savefig(output_path_figures+str(nuts_level)+'_'+nuts_interest_id+"_"+nuts_interest_name.replace('/', '_')+".png")
        plt.close()



In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_earth = 6371  # Radius of the Earth in kilometers
    distance = radius_earth * c

    return distance
  
stops['centroid_nuts'] = stops['nuts'].apply(lambda x: dict_centroid_pop.get(x))
stops['distance_nuts_pop_centr'] = stops.apply(lambda x: haversine(x.centroid_nuts[0], x.centroid_nuts[1],
                                                                   x.stop_lon, x.stop_lat) if x.centroid_nuts is not None else 0, axis=1)


In [ ]:
# Check for each station which regions can be reached based on the stops_times
l_stops = []
for gtfs in gtfs_paths:
    stop_times = pd.read_csv(gtfs['path']+'stop_times.txt')
    stop_times['stop_id'] = stop_times['stop_id'].apply(lambda x: gtfs['country']+str(x))
    stop_times['stop_id'] = stop_times['stop_id'].apply(lambda x: parent_stop_dict.get(x,x))
    stop_times['nuts'] = stop_times['stop_id'].apply(lambda x: dict_stations_nuts[x])
    l_stops += [stop_times]

stop_times = pd.concat(l_stops).reset_index(drop=True)


# Function to get unique NUTS regions after each stop within a trip
def get_nuts_for_stop(group):
    nuts_for_stop = {}
    for i, row in group.iterrows():
        # Get the NUTS3 regions after the current stop
        nuts_after_stop = group.loc[i:, 'nuts'].unique()
        nuts_for_stop[row['stop_id']] = set(nuts_after_stop)
    return nuts_for_stop

# Group by trip_id and apply the get_nuts_for_stop function
# It has for each trip which NUTs can be reached from which stop
nuts_for_stops = stop_times.groupby('trip_id').apply(get_nuts_for_stop).reset_index()




In [ ]:
# Compute nuts reachable from each station, number of trips per nuts
dict_nuts_reachable_station = {}
dict_number_trips_station = {}
dict_number_trips_station_to_nuts = {}
nuts_considered = set()

for dict_s_nuts in nuts_for_stops[0]:
    for station, nuts_reachable in dict_s_nuts.items():
        nuts_considered = nuts_considered.union(nuts_reachable)
        if station not in dict_nuts_reachable_station.keys():
            dict_nuts_reachable_station[station] = set()
        if station not in dict_number_trips_station.keys():
            dict_number_trips_station[station] = 0
        if station not in dict_number_trips_station_to_nuts.keys():
            dict_number_trips_station_to_nuts[station] = {}
            
        dict_nuts_reachable_station[station] = dict_nuts_reachable_station[station].union(nuts_reachable)
        dict_number_trips_station[station] += 1
        
        for n in nuts_reachable:
            if n not in dict_number_trips_station_to_nuts[station].keys():
                dict_number_trips_station_to_nuts[station][n] = 0
            dict_number_trips_station_to_nuts[station][n] += 1

dict_distance_centroid_pop_stop = dict(zip(stops['stop_id'], stops['distance_nuts_pop_centr']))

In [ ]:
## Filtering

# Pareto stations for nuts
stops['nuts_reachable'] = stops['stop_id'].apply(lambda x: dict_nuts_reachable_station.get(x))
stops['n_nuts_reachable'] = stops['nuts_reachable'].apply(lambda x: 0 if x is None else len(x))
stops['n_trips'] = stops['stop_id'].apply(lambda x: dict_number_trips_station.get(x, 0))
stops['n_trips_nuts'] = stops['stop_id'].apply(lambda x: dict_number_trips_station_to_nuts.get(x))

def is_dominated(df_stations, i):
    current_stop_id = df_stations['stop_id'].loc[i]
    current_nuts_reachable = df_stations['nuts_reachable'].loc[i]
    current_n_trips = df_stations['n_trips'].loc[i]
    if current_n_trips == 0:
        return True
    current_n_trips_nuts = df_stations['n_trips_nuts'].loc[i]
    current_dist_centre_nuts = df_stations['distance_nuts_pop_centr'].loc[i]

    for ic in df_stations.index:
        if ic != i:
            other_nuts_reachable = df_stations['nuts_reachable'].loc[ic]
            other_n_trips = df_stations['n_trips'].loc[ic]
            other_n_trips_nuts = df_stations['n_trips_nuts'].loc[ic]
            other_dist_centre_nuts = df_stations['distance_nuts_pop_centr'].loc[ic]
            if other_n_trips > 0:

                if current_nuts_reachable <= other_nuts_reachable:
                    # Can reach the same nuts as the other
                    current_trips_nuts = list(current_n_trips_nuts.keys())
                    i_n = 0
                    dominated = True
                    while (i_n < len(current_trips_nuts)) and dominated:
                        nuts_looking = current_trips_nuts[i_n]
                        if current_n_trips_nuts[nuts_looking] > other_n_trips_nuts[nuts_looking]:
                            dominated = False
                        i_n += 1
    
                    if dominated:   
                        # If we are here we checked all the nuts and we don't have fewer options
                        # we could have the same for all of them
                        all_equal = True
                        i_n = 0
                        while (i_n < len(current_trips_nuts)) and all_equal:
                            n = current_trips_nuts[i_n]
                            if current_n_trips_nuts[n] != other_n_trips_nuts[n]:
                                all_equal = False
                            i_n += 1
        
                        if not all_equal or current_dist_centre_nuts > other_dist_centre_nuts:
                            # If all the same check the distance w.r.t. centre nuts to keep only one
                            return True
                
    return False
    
    
    
dict_rail_stations_nuts = {}
for n in nuts_considered: #[x for x in nuts_considered if x.startswith('DE')]: #nuts_considered:
    stops_n = stops[stops.nuts==n].copy()
    stops_keep = []
    for stop_index in stops_n.index:
        if not is_dominated(stops_n, stop_index):
            stops_keep += [stops_n.loc[stop_index].stop_id]
        
    dict_rail_stations_nuts[n] = stops_keep
    


stations_kept = []
for s in dict_rail_stations_nuts.values():
    stations_kept += s

len(stations_kept)

In [ ]:
# Save results filtered
stops_kept = stops[stops.stop_id.isin(stations_kept)]
dict_aggreaged_rail = {}
for c in stops_kept.country.drop_duplicates():
    stops_kept[stops_kept.country==c]
    
    # Merge dataframes
    merged_df = stops_kept[stops_kept.country==c][['stop_id', 'stop_name', 'NUTS_ID']].merge(nuts_data[['NUTS_ID','NAME_LATN','LEVL_CODE']], on='NUTS_ID', how='left')
    
    # Group by NUTS_ID and aggregate data
    
    # Group by NUTS_ID and aggregate data
    aggregated_df = merged_df.groupby(['NUTS_ID', 'NAME_LATN', 'LEVL_CODE']).agg(
        num_rail_stations=('stop_id', 'nunique'),  # Count the number of unique rail stations
        rail_stations=('stop_id', lambda x: list(zip(x, merged_df.loc[merged_df['stop_id'].isin(x), 'stop_name'])))  # Construct the list of rail stations
    ).reset_index()   

    dict_aggreaged_rail[c] =  aggregated_df[['NUTS_ID', 'LEVL_CODE', 'NAME_LATN', 'LEVL_CODE', 'num_rail_stations', 'rail_stations']]

### Filter airports

In [ ]:
# Read flight schedules for a given day
df_fs = pd.read_parquet(flight_schedules_path)

# Keep airports from which flights depart and arrive
airports_w_flights = set(df_fs['origin']).union(df_fs['destination'])


filtered_airports = airports[airports.icao_id.isin(airports_w_flights)]

nuts_data['aiports_filtered'] = None
i_w_a = ~nuts_data['airports'].isnull()

nuts_data.loc[i_w_a, 'aiports_filtered'] = nuts_data.loc[i_w_a]['airports'].apply(lambda x: list(set(x).intersection(airports_w_flights)))

nuts_data['num_airports_filtered'] = nuts_data['aiports_filtered'].apply(lambda x: len(x) if x is not None else 0)


### Save results

In [ ]:
# Merge back airports and rail results
lraildf = []
for r in dict_aggreaged_rail.values():
    lraildf += [r]

rail_df_filtered = pd.concat(lraildf)

df = nuts_data[(nuts_data.CNTR_CODE.isin(countries)) & (nuts_data.LEVL_CODE==nuts_level)][['NUTS_ID','CNTR_CODE','LEVL_CODE','NAME_LATN','num_airports_filtered','aiports_filtered']].copy()


df = df.merge(rail_df_filtered[['NUTS_ID','num_rail_stations','rail_stations']], how='left', on='NUTS_ID')
df.loc[df.num_rail_stations.isnull(),'num_rail_stations'] = 0
df = df.rename({'aiports_filtered': 'airports', 'num_airports_filtered': 'num_airports'}, axis=1)
for c in countries:
    df[df.CNTR_CODE==c][['NUTS_ID','LEVL_CODE','NAME_LATN','num_rail_stations','num_airports',
                        'airports','rail_stations']].to_csv(output_path+'nuts'+str(nuts_level)+'_'+c+'_filtered.csv',index=False)



### Create html for visualisation

In [ ]:
# Import necessary libraries
import folium
from folium import plugins

# Sample grouped data for airports
df_f_g = df_fs.groupby('origin').count().reset_index()
dict_departures_airports = dict(zip(df_f_g['origin'], df_f_g['destination']))
df_f_g = df_fs.groupby('destination').count().reset_index()
dict_arrivals_airports = dict(zip(df_f_g['destination'], df_f_g['origin']))

your_latitude = 47.24
your_longitude = 3.28

# Create a map centered at a specific location
map_center = [your_latitude, your_longitude]  # Specify the center of the map
m = folium.Map(location=map_center, zoom_start=5)  # Adjust zoom level as needed

max_circle = 20
stops_kept = stops[stops.stop_id.isin(stations_kept)]
dict_factor = (max_circle / stops_kept.groupby('country').n_nuts_reachable.max()).to_dict()

# Create feature group for circle markers
dict_cmgr = {}
for c in stops_kept.country.drop_duplicates():
    dict_cmgr[c] = folium.FeatureGroup(name='Rail stations ' + c)

# Add NUTS regions with translucent filling
nuts_feature_group = folium.FeatureGroup(name='NUTS Regions')
for idx, row in nuts_data[(nuts_data.LEVL_CODE==nuts_level) & (nuts_data.CNTR_CODE.isin(countries))].iterrows():
    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style_function=lambda x: {'fillColor': 'orange', 'color': 'orange', 'fillOpacity': 0.2, 'weight': 3},
        tooltip=f"NUTS ID: {row['NUTS_ID']}"
    ).add_to(nuts_feature_group)

# Add the NUTS regions to the map
nuts_feature_group.add_to(m)


# Add circles for each stop with size proportional to NUTS reachable
for index, row in stops_kept.iterrows():
    stop_id = row['stop_id']
    stop_name = row['stop_name']
    n_nuts_reachable = row['n_nuts_reachable']
    n_trips = row['n_trips']
    lat = row['stop_lat']
    lon = row['stop_lon']
    nuts_id = row['NUTS_ID']
    country = row['country']
    
    # Calculate circle radius based on NUTS reachable
    radius = n_nuts_reachable * dict_factor[row['country']]  # Adjust multiplier as needed
    
    # Add circle marker to the map
    folium.CircleMarker(
        location=[lat, lon],
        radius=radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=f"{stop_name}<br>Nuts reachable: {n_nuts_reachable}<br>Num trips: {n_trips}<br>NUTS: {nuts_id}",
    ).add_to(dict_cmgr[country])

# Add feature group to the map
for c in dict_cmgr.values():
    c.add_to(m)

# Add airports
dict_cmga = {}
for c in countries:
    airports_c = set([item for sublist in list(df[(df.CNTR_CODE == c) & (df.airports.notnull())]['airports']) for item in sublist])
    ac = airports[airports.icao_id.isin(airports_c)]
    if len(ac) > 0:
        dict_cmga[c] = folium.FeatureGroup(name='Airports ' + c)
        
        for index, row in ac.iterrows():
            lat = row['lat']
            lon = row['lon']
            icao = row['icao_id']
            departures = dict_departures_airports.get(icao, 0)
            arrivals = dict_arrivals_airports.get(icao, 0)
            n_flights = departures + arrivals
            
            # Calculate circle radius based on PageRank centrality
            radius = max(4, n_flights * 0.01)
            
            # Add circle marker to the map
            folium.CircleMarker(
                location=[lat, lon],
                radius=radius,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=0.5,
                popup=f"{icao}<br>Departures: {departures}<br>Arrivals: {arrivals}",
            ).add_to(dict_cmga[c])

for c in dict_cmga.values():
    c.add_to(m)



# Add layer control with custom buttons
folium.LayerControl(collapsed=False).add_to(m)

m.save(output_path+'filtered_nuts_'+str(nuts_level)+'_with_nuts_'+'_'.join(countries)+'.html')

In [ ]:
dict_distance_matrix_nuts = {}

for c in countries:
    nuts_ids_c = nuts_data[(nuts_data.CNTR_CODE==c) & (nuts_data.LEVL_CODE==nuts_level)]['NUTS_ID']
    
    distance_matrix = np.zeros((len(nuts_ids_c), len(nuts_ids_c)))
    
    # Compute distances
    for i, nuts_id1 in enumerate(nuts_ids_c):
        for j, nuts_id2 in enumerate(nuts_ids_c):
            if i != j:
                lon1, lat1 = dict_centroid_pop[nuts_id1]
                lon2, lat2 = dict_centroid_pop[nuts_id2]
                distance_matrix[i, j] = haversine(lon1, lat1, lon2, lat2)
                if nuts_id2 == 'DEF0C':
                    pass #rint("HERE")
    
    # Convert the distance matrix to a DataFrame for easier handling
    dict_distance_matrix_nuts[c] = pd.DataFrame(distance_matrix, index=nuts_ids_c, columns=nuts_ids_c).sort_index(axis=0).sort_index(axis=1)


# Plot the heatmap of distances
import seaborn as sns
import matplotlib.pyplot as plt

for c in countries:
    plt.figure(figsize=(20, 20))
    sns.heatmap(dict_distance_matrix_nuts[c], fmt=".2f", cmap="YlGnBu")
    plt.title("Distance Matrix Heatmap "+c)
    plt.savefig(output_path_figures+c+"_"+str(nuts_level)+"_distance_pop_centroid_matrix.png")
    plt.close()
    

for c in countries:
    vmin_value = threshold_distance_plot_between_nuts
    vmax_value = threshold_distance_plot_between_nuts+0.001
    plt.figure(figsize=(20, 20))
    sns.heatmap(dict_distance_matrix_nuts[c], fmt=".2f", cmap="YlGnBu", vmin=vmin_value, vmax=vmax_value)
    plt.title("Distance Matrix Heatmap "+c+" threshold "+str(threshold_distance_plot_between_nuts)+"km")
    plt.savefig(output_path_figures+c+"_"+str(nuts_level)+"_distance_pop_centroid_matrix_threshold.png")
    plt.close()
    

# Save distance matrices
for c in countries:
    dict_distance_matrix_nuts[c].to_csv(output_path+'nuts'+str(nuts_level)+'_'+c+'_distance_pop_centroid_matrix.csv')